<img src="/home/vule/projects/learn_agent/images/basic_reflection.png" width="700"/>

In [1]:
cd ..

/home/vule/projects/learn_agent


In [2]:
%pprint

Pretty printing has been turned OFF


In [3]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

from src.key import openai_api_key


llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key)

In [4]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are an essay assistant tasked with writing excellent 2-paragraph essays."
     " Generate the best essay possible for the user's request."
     " If the user provides critique, respond with a revised version of your previous attempts."
     ),
     MessagesPlaceholder(variable_name="messages"),
])


prompt.format_messages(messages=[])

generate = prompt | llm


In [5]:
request = HumanMessage(content="Write an essay on why the little prince is relevant in modern childhood.")

essay = ""
# for chunk in generate.stream([request]):
#     print(chunk.content, end="")
#     essay += chunk.content

print("\n\n--------------\n\n")
print(essay)





--------------





In [6]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [7]:
reflection = reflect.invoke({"messages": [request, HumanMessage(content=essay)]})
print("\n\n--------------\n\n")
print(reflection)



--------------


content='**Critique of Essay Submission: "Why The Little Prince is Relevant in Modern Childhood"**\n\n**Overall Impression:**\nYour essay presents an interesting topic with the potential to resonate deeply with readers. However, the submission lacks depth and specificity in certain areas, and it could benefit from a more structured approach. Below are detailed critiques and recommendations to enhance your work.\n\n**1. Introduction:**\n- **Critique:** The introduction sets the stage for your topic but could be more engaging. It may lack a clear thesis statement that outlines the main points you will discuss.\n- **Recommendation:** Start with a compelling hook, perhaps a quote from "The Little Prince" or a brief anecdote about childhood. Clearly state your thesis, summarizing the key arguments you will explore regarding the relevance of the book in modern childhood.\n\n**2. Structure and Organization:**\n- **Critique:** The essay appears to be somewhat disorganized, w

In [8]:
reflection.content

'**Critique of Essay Submission: "Why The Little Prince is Relevant in Modern Childhood"**\n\n**Overall Impression:**\nYour essay presents an interesting topic with the potential to resonate deeply with readers. However, the submission lacks depth and specificity in certain areas, and it could benefit from a more structured approach. Below are detailed critiques and recommendations to enhance your work.\n\n**1. Introduction:**\n- **Critique:** The introduction sets the stage for your topic but could be more engaging. It may lack a clear thesis statement that outlines the main points you will discuss.\n- **Recommendation:** Start with a compelling hook, perhaps a quote from "The Little Prince" or a brief anecdote about childhood. Clearly state your thesis, summarizing the key arguments you will explore regarding the relevance of the book in modern childhood.\n\n**2. Structure and Organization:**\n- **Critique:** The essay appears to be somewhat disorganized, with ideas presented in a wa

In [9]:
# for chunk in generate.stream(
#     {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
# ):
#     print(chunk.content, end="")

print("\n\n--------------\n\n")
print(essay)



--------------





In [18]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langgraph.checkpoint.memory import MemorySaver

# define State
class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]

# define nodes/tools
def generation_node(state: AgentState):
    print('===============generation_node===============')
    essay = generate.invoke(state['messages'])
    essay.pretty_print()
    return {'messages': [HumanMessage(content=essay.content)]}
def reflection_node(state: AgentState):
    print('===============reflection_node===============')
    reflection = reflect.invoke(state['messages'])
    reflection.pretty_print()
    return {'messages': [HumanMessage(content=reflection.content)]}

# define graph
graph = StateGraph(AgentState)
graph.add_node("generate", generation_node)
graph.add_node("reflect", reflection_node)

def should_reflect(state: AgentState):
    print('===============should_reflect===============')
    if len(state['messages']) < 10:
        return True
    return False

graph.add_conditional_edges("generate", should_reflect, {True: "reflect", False: END})
graph.add_edge("reflect", "generate")
graph.set_entry_point("generate")

memory_saver = MemorySaver()
e_graph = graph.compile(checkpointer=memory_saver)


In [19]:
config = {"configurable": {"thread_id": "1"}}

In [20]:
request = HumanMessage(content="Write an essay on why learning is important.")


result = e_graph.invoke({"messages": [request]}, config)
print(result)

===============generation_node===============
================================== Ai Message ==================================

Learning is a fundamental aspect of human development and progress, influencing both individual growth and societal advancement. At its core, learning equips individuals with the knowledge and skills necessary to navigate the complexities of life. It fosters critical thinking, enhances problem-solving abilities, and promotes adaptability in an ever-changing world. The pursuit of education opens doors to opportunities, allowing individuals to engage more fully in their communities, contribute to the workforce, and make informed decisions that impact their lives and those around them. Furthermore, learning encourages a lifelong curiosity, motivating individuals to explore new ideas, cultures, and technologies, which is essential in our interconnected global society.

Beyond personal development, learning plays a pivotal role in shaping the future of society. An 

In [17]:
state = e_graph.get_state(config)
ChatPromptTemplate.from_messages(state.values["messages"]).pretty_print()

================================ Human Message =================================

Write an essay on why learning is important.

================================ Human Message =================================

Learning is a fundamental aspect of human development that underpins both personal and societal progress. It equips individuals with the knowledge and skills necessary to navigate the complexities of life and make informed decisions. From an early age, learning fosters critical thinking and problem-solving abilities, allowing individuals to adapt to changing circumstances and challenges. In a rapidly evolving world, where technological advancements and global interconnectedness are the norms, continuous learning is vital. It not only enhances one’s professional qualifications but also broadens perspectives, promoting empathy and understanding among diverse cultures and ideas. This lifelong journey of learning cultivates a sense of curiosity and encourages innovation, ultimately d